In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb



In [84]:
df = pd.read_csv('feat_games.csv')

- make unique id for each games as a combo of teamA and teamB's ids, with the season. Make this the index
- keep winTeamA, seed_difference, average ranking diff, early ranking, late ranking, median ranking, power6, offensive efficiency, defensive efficiency,  field goal attempts per game, points per game, opponent turnovers,  turnover pctg, and assists per game.
- scale data (minmax, and research other scaling methods)

In [85]:
columns = ['Season','Team A', 'Team B', 'WinTeamA', 'ScoreDiff', 'Seed_AmenoB', 'Early Ranking_AmenoB', 'Late Ranking_AmenoB', 'defEff_AmenoB', 'offEff_AmenoB', 'Power6_AmenoB', 'Median Ranking_AmenoB', 'Average Ranking_AmenoB', 'OppTO_AmenoB',
          'TotalPoints_AmenoB','TotalAssists_AmenoB', 'TOPctg_AmenoB']
df = df[columns]

In [86]:
features = ['Seed_AmenoB', 'Early Ranking_AmenoB', 'Late Ranking_AmenoB', 'defEff_AmenoB', 'offEff_AmenoB', 'Power6_AmenoB', 'Median Ranking_AmenoB', 'Average Ranking_AmenoB', 'OppTO_AmenoB',
          'TotalPoints_AmenoB','TotalAssists_AmenoB', 'TOPctg_AmenoB']

In [6]:
df.head(3)

,Season,Team A,Team B,WinTeamA,ScoreDiff,Seed_AmenoB,Early Ranking_AmenoB,Late Ranking_AmenoB,defEff_AmenoB,offEff_AmenoB,Power6_AmenoB,Median Ranking_AmenoB,Average Ranking_AmenoB,OppTO_AmenoB,TotalPoints_AmenoB,TotalAssists_AmenoB,TOPctg_AmenoB
0,2003,1411,1421,0,-8,0,29.890244,1.947368,-21.424483,2.653173,0,-21.062780,-5.152500,66.0,119,48,-0.852268
1,2003,1112,1436,1,29,-15,-175.866125,-163.227723,-12.873977,8.258702,0,-167.405295,-168.120756,174.0,420,82,-2.049504
2,2003,1113,1272,1,13,3,15.085366,11.226842,12.929152,3.754363,0,-12.730942,-1.107500,-61.0,42,-31,1.232274


In [87]:
df['UniqueKey'] = df.loc[:,'Season'].astype(str) + '_'+ df.loc[:,'Team A'].astype(str) + '_'+ df.loc[:,'Team B'].astype(str)
df.set_index('UniqueKey', inplace=True)

                                                                                           

In [88]:
df.drop(['Team A', 'Team B'], axis=1, inplace=True)

In [89]:
df.drop(['ScoreDiff'], axis=1, inplace=True)

In [90]:
df.head(3)

,Season,WinTeamA,Seed_AmenoB,Early Ranking_AmenoB,Late Ranking_AmenoB,defEff_AmenoB,offEff_AmenoB,Power6_AmenoB,Median Ranking_AmenoB,Average Ranking_AmenoB,OppTO_AmenoB,TotalPoints_AmenoB,TotalAssists_AmenoB,TOPctg_AmenoB
UniqueKey,,,,,,,,,,,,,,
2003_1411_1421,2003,0,0,29.890244,1.947368,-21.424483,2.653173,0,-21.062780,-5.152500,66.0,119,48,-0.852268
2003_1112_1436,2003,1,-15,-175.866125,-163.227723,-12.873977,8.258702,0,-167.405295,-168.120756,174.0,420,82,-2.049504
2003_1113_1272,2003,1,3,15.085366,11.226842,12.929152,3.754363,0,-12.730942,-1.107500,-61.0,42,-31,1.232274


Using standard scalar because all the features follow a normal distribution for the most part. We can check Robust scaling because some teams might have outliers in certain features

In [22]:
X = df[features]
y = df['WinTeamA']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)




In [23]:
from scipy.sparse import csr_matrix


In [24]:
# Assuming X_scaled is a sparse matrix
X_dense = X_scaled.toarray() if isinstance(X_scaled, csr_matrix) else X_scaled

# Create DMatrix with dense data
dmatrix = xgb.DMatrix(X_dense, label=y)

/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [100]:
params = {
    'objective': 'binary:logistic',  # Binary classification problem
    'max_depth': 3,                   # Maximum depth of a tree
    'eta': 0.02,                       # Learning rate
    'eval_metric': 'mae',          # Evaluation metric (logarithmic loss)
    'booster': 'gbtree',
    'seed': 42,
    'gamma': 10,
    'num_parallel_tree': 3,
    'min_child_weight': 40,
    'subsample': 0.35
}

First K_fold_cross_validation function

In [74]:


def k_fold_cross_validation2(X, y, model_params, n_splits=5, random_state=None):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    mae_scores = []  # To store mean squared error for each fold
    y_preds = []  # To store predictions for each fold
    y_testedOn = []  # To store true values for each fold
    n = 100

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dtrain_log = xgb.DMatrix(X_train, y_train)
        dtest_log = xgb.DMatrix(X_test, y_test)

        model = xgb.train(
           params=params,
           dtrain=dtrain_log,
           num_boost_round=n,
        )

        preds = model.predict(dtest_log)
        threshold = 0.5
        binary_preds = [1 if prob > threshold else 0 for prob in preds]
        
        mae = mean_absolute_error(y_test, binary_preds)
        mae_scores.append(mae)
        y_preds.extend(binary_preds)
        y_testedOn.extend(y_test)

    return mae_scores, y_preds, y_testedOn


In [75]:
kf = k_fold_cross_validation(X_scaled, y, model_params=params, random_state=42)


/var/folders/_0/lfzv1d311sjbpc94z2sh41m80000gn/T/ipykernel_27768/4092677705.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/var/folders/_0/lfzv1d311sjbpc94z2sh41m80000gn/T/ipykernel_27768/4092677705.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/opt/homebrew/Caskr

[13:50:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[13:50:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[13:50:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[13:50:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[13:50:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.



/var/folders/_0/lfzv1d311sjbpc94z2sh41m80000gn/T/ipykernel_27768/4092677705.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/var/folders/_0/lfzv1d311sjbpc94z2sh41m80000gn/T/ipykernel_27768/4092677705.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/opt/homebrew/Caskr

In [76]:
mae_scores = kf[0]

average_mae = np.mean(mae_scores)
print(f'Average Mean Absolute Error: {average_mae}')

Average Mean Absolute Error: 0.2938496746048773


In [81]:
print(set(df.Season))

{2016, 2017, 2018, 2019, 2021, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015}


look at this notebook for help on xgboost implementation: https://www.kaggle.com/code/publicmikewang/march-madness-model-prediction/notebook

look at this https://www.kaggle.com/code/tihonby/march-madness-3th-place-solution

Second k-fold cross validation function    
difference is i am making my rounds of training based on the seasons.  For every season I am doing a training and testing set exlucing the season from the training set and making the testing set the sample of the data with the season=season.

In [126]:
params = {
    'objective': 'binary:logistic',  # Binary classification problem
    'max_depth': 3,                   # Maximum depth of a tree
    'eta': 0.02,                       # Learning rate
    'eval_metric': 'mae',          # Evaluation metric (logarithmic loss)
    'booster': 'gbtree',
    'seed': 42,
    'gamma': 10,
    'num_parallel_tree': 3,
    'min_child_weight': 40,
    'subsample': 0.35
}

features = ['Seed_AmenoB', 'Early Ranking_AmenoB', 'Late Ranking_AmenoB', 'defEff_AmenoB', 'offEff_AmenoB', 'Power6_AmenoB', 'Median Ranking_AmenoB', 'Average Ranking_AmenoB', 'OppTO_AmenoB',
          'TotalPoints_AmenoB','TotalAssists_AmenoB', 'TOPctg_AmenoB']

In [141]:
def cv_xg_boost(df, features, params):
    models = {}
    ssn_mae_scores = []
    ssn_binary_predictions = []
    ssn_targets = []
    scaler = MinMaxScaler()
    num_rounds = 500
    features = sorted(features) # did this to  try and debug the error I am on right now 
    
    
    for ssn in sorted(set(df.Season)):
        x_train = df.loc[df.Season!=ssn, features].values
        y_train = df.loc[df.Season!=ssn, 'WinTeamA'].values   
        x_val = df.loc[df.Season==ssn, features].values
        y_val = df.loc[df.Season==ssn, 'WinTeamA'].values
        x_train = scaler.fit_transform(x_train) # scaling training set X
        x_val = scaler.fit_transform(x_val) # scaling testing set X
        dtrain = xgb.DMatrix(x_train, label = y_train)
        dval = xgb.DMatrix(x_val, label = y_val)
        models[ssn] = xgb.train(
              params = params,
              dtrain = dtrain,
              num_boost_round = num_rounds,
        )
    
        preds = models[ssn].predict(dval)
        binary_preds = [1 if pred > 0.5 else 0 for pred in preds]
        print(f'season {ssn} mae: {mean_absolute_error(y_val, binary_preds)}')
        ssn_mae_scores.append(mean_absolute_error(y_val, binary_preds))
        ssn_binary_predictions += list(binary_preds)
        ssn_targets += list(y_val)
    
    print(f'Average mae: {np.mean(ssn_mae_scores)}')
    # errors start here
    features = sorted(features)# did this to  try and debug the error I am on right now 
    
    feature_importance = {feature: 0 for feature in features}
    for ssn in models:
        importance = models[ssn].get_fscore()
        for feature in importance:
            feature_importance[feature] += importance[feature]
    
    # Sort features by importance
    sorted_feature_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    features, importances = zip(*sorted_feature_importance)
    
    # Plot feature importances
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(features)), importances, align='center')
    plt.yticks(range(len(features)), features)
    plt.xlabel('Feature Importance')
    plt.ylabel('Features')
    plt.title('XGBoost Feature Importance')
    plt.show()

    return models

In [142]:
cv_xg_boost(df, features, params)

season 2003 mae: 0.296875
season 2004 mae: 0.328125
season 2005 mae: 0.265625
season 2006 mae: 0.3125
season 2007 mae: 0.203125
season 2008 mae: 0.3125
season 2009 mae: 0.234375
season 2010 mae: 0.328125
season 2011 mae: 0.4626865671641791
season 2012 mae: 0.417910447761194
season 2013 mae: 0.3582089552238806
season 2014 mae: 0.3582089552238806
season 2015 mae: 0.34328358208955223
season 2016 mae: 0.40298507462686567
season 2017 mae: 0.2835820895522388
season 2018 mae: 0.373134328358209
season 2019 mae: 0.31343283582089554
season 2021 mae: 0.3484848484848485
Average mae: 0.33017598246143026


KeyError: 'f0'

In [ ]:
xgb.plot_importance(xgb_clf)
plt.figure(figsize = (16, 12))
plt.show()